
#  Quantum network knowledge graphs

El propósit de este documento es crear un pipelien de creación de un grafo de personas asociadas en la comunidad de comptuación cuántica, ya fuera pro mención o por perfilamiento.

Nos interesa modelar el siguente grafo de conocimiento, el ecosistema de empresas e investigadores, así como las perosnas en el ecosistema que investigan, y que están en linkedin, si la idea es generar covnersación sobre quantum, la investigación, temas de investigación en común, deberían tener un mapa para poder orientar los esfuerzos.

In [19]:
import pandas as pd
from neo4j import GraphDatabase, Driver, Session
from pydantic import BaseModel
from contextlib import contextmanager
from typing import Optional, List, Dict, Any, Type

from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

from dotenv import load_dotenv, find_dotenv
import os


_ = load_dotenv(find_dotenv())
_

True

- Calificaremos algunas preguntas para darle peso a alguans relaciones y atibutos.
- Interés principal en Computación Cuántica - (Seleccionar una o más): (Tags Que serán Sub Nodos)
- ¿Ha trabajado previamente con tecnologías cuánticas?
- ¿Qué espera obtener de este evento?:


In [20]:
def create_driver(uri: str, user: str, password: str) -> Driver:
    """
    Crea un driver de conexión a Neo4j
    
    Args:
        uri: URI de conexión (ej: "bolt://localhost:7687")
        user: Usuario de Neo4j
        password: Contraseña
        
    Returns:
        Driver de Neo4j
    """
    return GraphDatabase.driver(uri, auth=(user, password))


def close_driver(driver: Driver) -> None:
    """Cierra la conexión del driver"""
    driver.close()


@contextmanager
def get_session(driver: Driver):
    """
    Context manager para manejar sesiones de Neo4j
    
    Usage:
        with get_session(driver) as session:
            # usar session
    """
    session = driver.session()
    try:
        yield session
    finally:
        session.close()

In [22]:
driver = create_driver(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_QUANTUM_NETWORK_AURA")
)

In [ ]:

    
with get_session(driver) as session:
    result = session.run("""
    MATCH (n) RETURN n
    """)
    result.single()

AuthError: {neo4j_code: Neo.ClientError.Security.Unauthorized} {message: The client is unauthorized due to authentication failure.} {gql_status: 42NFF} {gql_status_description: error: syntax error or access rule violation - permission/access denied. Access denied, see the security logs for details.}

In [ ]:

def data_enhancer():
    '''
    Esta función tiene como propósito crear un LLM que puede extraer de un determinado texto, de 


    '''
    llm = ChatOllama(
        model="llama3.2:3b",
        base_url="http://localhost:11434",
        temperature=0.0
    )

    return llm

llm = data_enhancer()
llm.invoke("1+1")

    

AIMessage(content='1 + 1 = 2', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-11-16T21:25:27.1896229Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5045879500, 'load_duration': 4835097500, 'prompt_eval_count': 28, 'prompt_eval_duration': 40363600, 'eval_count': 8, 'eval_duration': 148681200, 'logprobs': None, 'model_name': 'llama3.2:3b', 'model_provider': 'ollama'}, id='lc_run--422de324-3e67-4e44-8add-e58a7f9fe7f4-0', usage_metadata={'input_tokens': 28, 'output_tokens': 8, 'total_tokens': 36})

In [ ]:
'''

UNWIND rows
MERGE (p:Person)
ON CREATE SET
    p.name = rows.name
    p.location = rows.location
    p.industry = rows.industry
    p.interests = rows.interests
    P.linkedin_url = rows.linkedin_url

MERGE (o:Organization)
ON CREATE SET
    o.name = o.organization

WITH p, o
MERGE (p)-[:WORKS_AT]->(o)

MERGE (d:Domain)
ON CREATE SET
    i.name = rows.domain
MERGE (p)-[:HAS_INTEREST]->(d)




'''

In [ ]:
def normalize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normaliza los nombres de columnas del CSV a nombres más manejables.
    
    Mapeo:
    - 'Nombre completo' -> 'name'
    - 'Correo electrónico' -> 'email'
    - 'Organización / Empresa' -> 'organization'
    - 'Cargo / Rol' -> 'role'
    - 'Sector al que pertenece su organización' -> 'industry_sector'
    - 'Interés principal en Computación Cuántica - (Seleccionar una o más)' -> 'interests'
    - '¿Ha trabajado previamente con tecnologías cuánticas?' -> 'quantum_experience'
    - '¿Qué espera obtener de este evento?' -> 'event_expectations'
    - 'LinkedIn' -> 'linkedin_url'
    
    Args:
        df: DataFrame con columnas originales
        
    Returns:
        DataFrame con columnas normalizadas
    """
    column_mapping = {
        'Nombre completo': 'name',
        'Correo electrónico': 'email',
        'Organización / Empresa': 'organization',
        'Cargo / Rol': 'role',
        'Sector al que pertenece su organización': 'industry_sector',
        'Interés principal en Computación Cuántica - (Seleccionar una o más)': 'interests',
        '¿Ha trabajado previamente con tecnologías cuánticas?': 'quantum_experience',
        '¿Qué espera obtener de este evento?': 'event_expectations',
        'LinkedIn': 'linkedin_url',
        'Timestamp': 'timestamp'  # Mantener timestamp para referencia
    }
    
    # Renombrar solo las columnas que existen
    df_normalized = df.rename(columns={k: v for k, v in column_mapping.items() if k in df.columns})
    
    return df_normalized

In [ ]:

def clean_text(value: Any) -> Optional[str]:
    """
    Limpia valores de texto: elimina espacios extra, convierte a string,
    maneja NaN y valores vacíos.
    
    Args:
        value: Valor a limpiar
        
    Returns:
        String limpio o None si está vacío
    """
    if pd.isna(value) or value == '':
        return None
    
    # Convertir a string y limpiar
    text = str(value).strip()
    
    # Si después de limpiar está vacío, retornar None
    if text == '' or text.lower() == 'nan':
        return None

In [ ]:
dataframe = pd.read_csv('../data/quantum_network.csv')
dataframe.head()

,Timestamp,Nombre completo,Correo electrónico,Organización / Empresa,Cargo / Rol,Sector al que pertenece su organización,Interés principal en Computación Cuántica - (Seleccionar una o más),¿Ha trabajado previamente con tecnologías cuánticas?,¿Qué espera obtener de este evento?,Preferencia de comida (para el lunch de cortesía),¿Desea recibir novedades e invitaciones a futuros eventos de IBM y QNOW?,Email confirmación,LinkedIn,Column 12
0,online,Federico Holik,holik@fisica.unlp.edu.ar,UNLP - IFLP,Investigador,Academia / Investigación,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,online,Mariano Caruso,mcaruso@fidesol.org,FIDESOL,Investigador & Desarrollo,Industria / Empresa privada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Tomás Tagliani,tomas@falcondale.pro,Falcondale,Founder,Startup / Emprendimiento,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Julio Cella,Julio.Cella@ibm.com,IBM,Security Client Leader,Industria / Empresa privada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Victoria Dominguez,marketing@qnow.tech,QNOW,Marketing,Startup / Emprendimiento,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# PERSON NODE
# Nombre completo --> name
# Correo electrónico-->email
# ¿Ha trabajado previamente con tecnologías cuánticas?
# Cargo / Rol --> role
# LinkedIn --> linkedin_url

# DOMAIN
# Interés principal en Computación Cuántica - (Seleccionar una o más)


# ORGANIZATION NODE
# Organización / Empresa --> organtization
# Sector al que pertenece su organización --> industry_sector

 